<a href="https://colab.research.google.com/github/bongomarcel/CERI_Pandas/blob/main/Self%20Study%20-%20Pandas%20Efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Examples of processing data efficiently with Pandas

To use Pandas efficiently there are common mistakes that should be avoided.

Pandas is built on top of the NumPy array structure and most operations are carries out in C compiled modules (either through NumPy or Pandas' own Python extensions written in Cython and compiled to C). Source: https://realpython.com/fast-flexible-pandas/

The paradox is that what may otherwise “look like” Pythonic code can be suboptimal in Pandas as far as efficiency is concerned. You have to think in terms of vectorised operations and not in terms of individual cell or per-row operations.

That being said, rather write clean code than over-optimized code! Pandas is designed to be fast if used correctly.

This tutorial will cover:
- Advantages of using `datetime` data with time series data
- The most efficient route to doing batch calculations
- Saving time by storing data with HDFStore

After loading the data, you’ll successively progress through more efficient ways to get to the end result. You will see that there are many ways to skin a cat using Pandas. However, not all of the available options will scale equally well to larger, more demanding datasets.

In [1]:
! git clone https://github.com/bongomarcel/CERI_Pandas.git

Cloning into 'CERI_Pandas'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 93 (delta 38), reused 45 (delta 9), pack-reused 0
Receiving objects: 100% (93/93), 10.80 MiB | 13.74 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [4]:
%cd CERI_Pandas

/content/CERI_Pandas


In [2]:
import numpy as np
import pandas as pd

The goal of this example will be to apply time-of-use energy tariffs to find the total cost of energy consumption for one year. That is, at different hours of the day, the price for electricity varies, so the task is to multiply the electricity consumed for each hour by the correct price for the hour in which it was consumed.

The data has two columns: one for date plus time and one for electrical energy consumed in kilowatt hours.

In [5]:
df = pd.read_csv("./data/demand_profile.csv")
df.head()

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 1:00,0.580
2,1/1/13 2:00,0.572
3,1/1/13 3:00,0.596
4,1/1/13 4:00,0.592


## Datetime conversion
This looks okay at first glance, but there's a small issue. Pandas and NumPy have a concept of dtypes (data types). If no arguments are specified, `date_time` will take on an `object` dtype:

In [6]:
df.dtypes

date_time      object
energy_kwh    float64
dtype: object

In [7]:
type(df.iat[0, 0]) # iat: Access a single value for a row/column pair by integer position.

str

The problem is that it is inefficient to work with dates as strings (also memory inefficient). For working with time series data, you'll want the `date_time` column to be formatted as an array of `datetime` objects. (Pandas calls this a Timestamp.)

In [8]:
df['date_time'] = pd.to_datetime(df['date_time'])
df['date_time'].dtype

dtype('<M8[ns]')

The dataframe now has two columns and a numerical index for referencing the rows.

In [9]:
df.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 01:00:00,0.580
2,2013-01-01 02:00:00,0.572
3,2013-01-01 03:00:00,0.596
4,2013-01-01 04:00:00,0.592


The code below will be used to time the execution time of the following functions. It can be used as a decorator.

In [10]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r took: %2.4f sec' % (f.__name__, te-ts))
        return result
    return wrap

In [11]:
@timing
def convert(df, column_name):
  return pd.to_datetime(df[column_name])

#Note: Need to read df in again to have object dtype again
df = pd.read_csv("./data/demand_profile.csv")
convert(df, 'date_time')


func:'convert' took: 0.7187 sec


0      2013-01-01 00:00:00
1      2013-01-01 01:00:00
2      2013-01-01 02:00:00
3      2013-01-01 03:00:00
4      2013-01-01 04:00:00
               ...        
8755   2013-12-31 19:00:00
8756   2013-12-31 20:00:00
8757   2013-12-31 21:00:00
8758   2013-12-31 22:00:00
8759   2013-12-31 23:00:00
Name: date_time, Length: 8760, dtype: datetime64[ns]

On the default Google Colab instance it takes just under a second to convert the datetime strings to datetime objects. Clearly not fast enough, given that time series data often comes in fine resolution. Considering the electricity demand data, for example, it is not unreasonable to anticipate a problem where the resolution is on a minute-basis and the time series range is more than, say, 20 years for all municipalities in South Africa.

We can speed things up by reducing the amount of data that needs to be interpreted by Pandas. If the datetimes strings provided are consistend (often they are not!), then you can provide format parameter to specify how that strings should be parsed. The various `strftime` codes can be found [here](https://strftime.org/).

In [12]:
@timing
def convert_with_format(df, column_name):
  return pd.to_datetime(df[column_name], format='%d/%m/%y %H:%M')

convert_with_format(df, 'date_time')

func:'convert_with_format' took: 0.0440 sec


0      2013-01-01 00:00:00
1      2013-01-01 01:00:00
2      2013-01-01 02:00:00
3      2013-01-01 03:00:00
4      2013-01-01 04:00:00
               ...        
8755   2013-12-31 19:00:00
8756   2013-12-31 20:00:00
8757   2013-12-31 21:00:00
8758   2013-12-31 22:00:00
8759   2013-12-31 23:00:00
Name: date_time, Length: 8760, dtype: datetime64[ns]

This conversion should execute in about 0.025 seconds which is a big improvement.
### Some technical details to keep in mind:
If data is only imported once or infrequently, one can parse dates as part of I/O step of the pipeline. Look at the docs for [`pandas.read_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html):  
* `parse_dates` let you specify which columns should be interpreted and parsed.
* `infer_datetime_format` is a boolean parameter (default = False). If True and parse_dates is enabled, pandas will attempt to infer the format of the datetime strings in the columns, and if it can be inferred, switch to a faster method of parsing them. In some cases this can increase the parsing speed by 5-10x.
* `date_parser` parameter allows you to apply a parsing function. The default is `dateutil.parser.parser`. Use this if the data is very noisy and contains a mixture of timezones.

## Looping over Pandas Data
Now that your dates and times are in a convenient format, let's calculate the electricity costs based on the cost factor assocaited with each hour of the day. The time-of-use costs will be defined as:

| Tariff Type | ZAR/kWh | Time Range |
| --- | ---: | --- |
| Peak | 2.50 | 17h00 to 24h00 |
| Shoulder | 2.20 | 7h00 to 17h00 |
| Off-Peak | 1.80 | 0h00 to 7h00|

If the cost of electricity was calculated based on a flat rate of R2.20, we could simply do:

In [13]:
df['cost'] = df['energy_kwh'] * 2.20
df.head()

,date_time,energy_kwh,cost
0,1/1/13 0:00,0.586,1.2892
1,1/1/13 1:00,0.580,1.2760
2,1/1/13 2:00,0.572,1.2584
3,1/1/13 3:00,0.596,1.3112
4,1/1/13 4:00,0.592,1.3024


However, our costs are conditional on the time of day. This is where it is tempting to by write a loop to do the conditional calculation.

### The Ugly

We will start from a less-than-ideal baseline solution and incrementally improve the implementation towards a Pythonic solution that fully leverages Pandas.

> Don't think in loops! Forget the C++ or Java way of doing things if you are using Pandas.

Let’s look at a loop approach that is not very Pythonic by creating a function to apply the appropriate tariff to a given hour:

In [14]:
def apply_tariff(kwh, hour):
  """Calculates cost of electricity for given hour."""
  if 0 <= hour < 7:
    rate = 1.8
  elif 7 <= hour < 17:
    rate = 2.2
  elif 17 <= hour < 24:
    rate = 2.5
  else:
    raise ValueError(f'Invalid hour: {hour}')
  return rate * kwh

In [15]:
# NOTE: Don't do this!
@timing
def apply_tariff_loop(df):
  """Calculate costs in loop.  Modifies `df` inplace."""
  energy_cost_list = []
  for i in range(len(df)):
    # Get electricity used and hour of day
    energy_used = df.iloc[i]['energy_kwh']
    hour = df.iloc[i]['date_time'].hour
    energy_cost = apply_tariff(energy_used, hour)
    energy_cost_list.append(energy_cost)
  df['cost'] = energy_cost_list

df = pd.read_csv("./data/demand_profile.csv")
df['date_time'] = convert_with_format(df, 'date_time')
apply_tariff_loop(df)
df.head()

func:'convert_with_format' took: 0.0276 sec
func:'apply_tariff_loop' took: 1.6823 sec


,date_time,energy_kwh,cost
0,2013-01-01 00:00:00,0.586,1.0548
1,2013-01-01 01:00:00,0.580,1.0440
2,2013-01-01 02:00:00,0.572,1.0296
3,2013-01-01 03:00:00,0.596,1.0728
4,2013-01-01 04:00:00,0.592,1.0656


The above can be seen as an antipattern in Pandas for several reasons:
1. It needs to initialize a list in which the outputs will be recorded.
2. It uses the opaque object `range(0, len(df))` to loop over.
3. after applying `apply_tariff()`, it has to append the result to a list that is used to make the new DataFrame column.
4. It performs chained indexing with `df.iloc[i]['date_time']`, which often leads to unintended results.
5. It takes more than 2 seconds to iterate over 8k rows!

### The Bad
The Bad is using Pandas' helper methods for looping: `DataFrame.itertuples()` and `DataFrame.iterrows()`. These are both generator methods that yield one row at a time.

`.itertuples()` yields a namedtuple for each row, with the row’s index value as the first element of the tuple.

`.iterrows()` yields pairs (tuples) of (index, Series) for each row in the DataFrame.

In [16]:
@timing
def apply_tariff_iterrows(df):
  energy_cost_list = []
  for index, row in df.iterrows():
    # Get electricity used and hour of day
    energy_used = row['energy_kwh']
    hour = row['date_time'].hour
    # Append cost list
    energy_cost = apply_tariff(energy_used, hour)
    energy_cost_list.append(energy_cost)
  df['cost_cents'] = energy_cost_list

df = pd.read_csv("./data/demand_profile.csv")
df['date_time'] = convert_with_format(df, 'date_time')
apply_tariff_iterrows(df)
df.head()

func:'convert_with_format' took: 0.0522 sec
func:'apply_tariff_iterrows' took: 0.7057 sec


,date_time,energy_kwh,cost_cents
0,2013-01-01 00:00:00,0.586,1.0548
1,2013-01-01 01:00:00,0.580,1.0440
2,2013-01-01 02:00:00,0.572,1.0296
3,2013-01-01 03:00:00,0.596,1.0728
4,2013-01-01 04:00:00,0.592,1.0656


The syntax is more slightly better (more explicit), and there is less clutter in the row value references. It's also almost 5x quicker!

The __Bad__ is that we are still using a Python `for`-loop which means each function call is done in Python.

### The Okay
You can further improve this operation using the `.apply()` method instead of `.iterrows()`. Pandas' `.apply()` method takes functions (callables) and applies them along an axis of a DataFrame (all rows, or all columns). In this example, a `lambda` function will help you pass the two columns of data into `apply_tariff()`:

In [17]:
@timing
def apply_tariff_withapply(df):
  df['cost'] = df.apply(
    lambda row: apply_tariff(
      kwh=row['energy_kwh'],
      hour=row['date_time'].hour),
    axis=1)

df = pd.read_csv("./data/demand_profile.csv")
df['date_time'] = convert_with_format(df, 'date_time')
apply_tariff_withapply(df)
df.head()

func:'convert_with_format' took: 0.0274 sec
func:'apply_tariff_withapply' took: 0.1817 sec


,date_time,energy_kwh,cost
0,2013-01-01 00:00:00,0.586,1.0548
1,2013-01-01 01:00:00,0.580,1.0440
2,2013-01-01 02:00:00,0.572,1.0296
3,2013-01-01 03:00:00,0.596,1.0728
4,2013-01-01 04:00:00,0.592,1.0656


The syntactic advantages of `.apply()` are clear, with a significant reduction in the number of lines and very readable, explicit code. In this case, the time taken was roughly half that of the `.iterrows()` method.

However, this is not yet "blazingly fast". One reason is that `.apply()` will try internally to loop over Cython iterators. But in this case, the lambda that you passed isn't something that can be handled in Cython, so it's called in Python, which is consequently not all that fast.

If you were to use `.apply()` for hourly data and 400 sites this computation would still take about 15 minutes of processing time. If this calculation is intended to be a small part of a larger model, you'd really want to speed things up. That's where vectorized operations come in handy.

### The Good
__Selecting Data With `.isin()`__
The earlier used operation for a flat-rate cost calculation (df['energy_kwh'] * 28) was an example of a vectorized operation. This is the fastest way to do things in Pandas.

But how can you apply condition calculations as vectorized operations in Pandas? The trick is to first do a __select operation__ to group parts the DataFrame based on your conditions and then apply a __vectorized operation__ to each selected group.

In the example below, the rows are selected with Pandas' `.isin()` method and then apply the appropriate tariff in a vectorized operation.

Before you do this, it will make things a little more convenient if you set the date_time column as the DataFrame’s index:

In [18]:
@timing
def apply_tariff_isin(df):
  # Define hour range Boolean arrays
  peak_hours = df.index.hour.isin(range(17, 24))
  shoulder_hours = df.index.hour.isin(range(7, 17))
  off_peak_hours = df.index.hour.isin(range(0, 7))

  # Apply tariffs to hour ranges
  df.loc[peak_hours, 'cost_cents'] = df.loc[peak_hours, 'energy_kwh'] * 2.5
  df.loc[shoulder_hours,'cost_cents'] = df.loc[shoulder_hours, 'energy_kwh'] * 2.2
  df.loc[off_peak_hours,'cost_cents'] = df.loc[off_peak_hours, 'energy_kwh'] * 1.8

df = pd.read_csv("./data/demand_profile.csv")
df['date_time'] = convert_with_format(df, 'date_time')
df.set_index('date_time', inplace=True)
apply_tariff_isin(df)

func:'convert_with_format' took: 0.0620 sec
func:'apply_tariff_isin' took: 0.0151 sec


Notes:
* `isin()` returns an array of Boolean values.
* the `loc` indexer uses these get a slice of the DataFrame that only includes rows that match the correct hours.
* Then the slices are multipled by the appropriate tariff.

Bottome line: reduction of lines of code. 300x computational speedup over looping approach. 25x faster than `.apply()` approach.

### The Best

In `apply_tariff_isin()`, we are still doing some "manual work" by calling `df.loc` and `df.index.hour.isin()` three times. You could argue that this solution isn't scalable if we had a more granular range of time slots. (E.g., hourly rate fluctuations would require 24 .isin() calls.) Luckily, one can do things more programmatically with Pandas' `pd.cut()` function:

In [19]:
@timing
def apply_tariff_cut(df):
  cents_per_kwh = pd.cut(x=df.index.hour,
                         bins=[0, 7, 17, 24],
                         include_lowest=True,
                         labels=[1.8, 2.2, 2.5]).astype(float)
  print(cents_per_kwh)
  df['cost_cents'] = cents_per_kwh * df['energy_kwh']

df = pd.read_csv("./data/demand_profile.csv")
df['date_time'] = convert_with_format(df, 'date_time')
df.set_index('date_time', inplace=True)
apply_tariff_cut(df)
df

func:'convert_with_format' took: 0.0295 sec
[1.8 1.8 1.8 ... 2.5 2.5 2.5]
func:'apply_tariff_cut' took: 0.0080 sec


,energy_kwh,cost_cents
date_time,,
2013-01-01 00:00:00,0.586,1.0548
2013-01-01 01:00:00,0.580,1.0440
2013-01-01 02:00:00,0.572,1.0296
2013-01-01 03:00:00,0.596,1.0728
2013-01-01 04:00:00,0.592,1.0656
...,...,...
2013-12-31 19:00:00,0.450,1.1250
2013-12-31 20:00:00,0.543,1.3575
2013-12-31 21:00:00,1.135,2.8375


Notes:
* `pd.cut` is used to group a continuous variable into bins (i.e., categorial variable).
* `include_lowest` parameter indicates whether the first interval should be left-inclusive or not.
* This is a fully vectorized solution and it should be the fastest.
* Cut returns a categorical series which is not designed to be used for arithmetics since it could be things like 'Cat' or 'Dog'. And what does 'Dog' + 1 mean? Therefore, we need to cast all elements in the series to floats explicitly.

###The Overkill
The final option is to use NumPy functions to manipulate the underlying NumPy arrays for each DataFrame and then to integrate the results back into a Pandas data structures. This is possible because Pandas is built ontop of the NumPy library and works seamlessly with NumPy arrays and operations.

NumPy's `digitize()` function is similar to Pandas' `cut()` function. It bins the data but the bins are represented by an array of indexes representing which bin each hour belongs to.

In [20]:
@timing
def apply_tariff_digitize(df):
  prices = np.array([1.8, 2.2, 2.5])
  bins = np.digitize(df.index.hour.values, bins=[7, 17, 24])
  df['cost'] = prices[bins] * df['energy_kwh'].values
  #print('bins:', bins)
  #print('prices[bins]:', prices[bins])
  #print(df['energy_kwh'].values)

df = pd.read_csv("./data/demand_profile.csv")
df['date_time'] = convert_with_format(df, 'date_time')
df.set_index('date_time', inplace=True)
apply_tariff_digitize(df)

func:'convert_with_format' took: 0.0319 sec
func:'apply_tariff_digitize' took: 0.0022 sec


## Summary
| Method/Approach | Approximate Runtime (seconds) |
|:---|---:|
|`apply_tariff_loop()` | 3.00 |
|`apply_tariff_iterros()` | 0.400 |
|`apply_tariff_withapply()` | 0.020 |
|`apply_tariff_isin()` | 0.010 |
|`apply_tariff_cut()` | 0.005 |
|`apply_tariff_digitize()` | 0.002 |

### Saving intermediate results using HDFStore

Often when you build complex data models, you need to do some (once-off) pre-processing of your data. For example, if you had 10 years of minute-frequency electricity consumption data the date and time conversion might take 20 minutes, even if you specify the format parameter. You do not want to compute this every time you run your model during testing or analysis.

If you were to save as CSV, you would simply lose your datetime objects and have to re-process it when accessing again. Pandas has a built-in solution for this which uses HDF5, a high-performance storage format designed specifically for storing tabular arrays of data. It is also highly fault tolerant and designed to work on cheap hardware. Pandas' HDFStore class allows you to store your DataFrame in an HDF5 file so that it can be accessed efficiently, while still retaining column types and other metadata. It is a dictionary-like class, so you can read and write just as you would for a Python `dict` object.

In [ ]:
# Create storage object with filename `processed_data`
data_store = pd.HDFStore('processed_data.h5')

# Put DataFrame into the object setting the key as 'preprocessed_df'
data_store['preprocessed_df'] = df
data_store.close()

# Access data store
data_store = pd.HDFStore('processed_data.h5')

# Retrieve data using key
preprocessed_df = data_store['preprocessed_df']
data_store.close()